In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import lightgbm as lgb
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [17]:
data = pd.read_csv('/Users/jeremiahwilson/Downloads/ads/reduced_feature_ads.csv')

In [18]:
new_data = data.fillna(0)

In [19]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)
new_data = new_data.drop(['Ad Name', 'Reporting Starts', 'Reporting Ends', 
                          'Ad Set Budget', 'Ad Set Budget Type', 'Result Indicator'], axis = 1)

In [20]:
train = new_data.drop('Results',axis=1)
target = new_data.Results

In [21]:
# Feature importance

#lightGBM model fit
gbm = lgb.LGBMRegressor()
gbm.fit(train, target)
gbm.booster_.feature_importance()

# importance of each attribute
fea_imp_ = pd.DataFrame({'cols':train.columns, 'fea_imp':gbm.feature_importances_})
fea_imp_.loc[fea_imp_.fea_imp > 0].sort_values(by=['fea_imp'], ascending = False)

,cols,fea_imp
1,Amount Spent (USD),881
2,Link Clicks,639
0,Reach,608
3,"CPM (Cost per 1,000 Impressions) (USD)",439
4,Unique CTR (Link Click-Through Rate),433


In [22]:
x = new_data.drop('Results', axis = 1)
y = new_data.Results

In [24]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.3)

In [25]:
train_data=lgb.Dataset(x_train,label=y_train)

In [26]:
param = {'num_leaves':110, 'objective':'regression','max_depth':7,'learning_rate':.05,'max_bin':200}
num_round=75
lgbm=lgb.train(param,train_data,num_round)

In [27]:
ypred2=lgbm.predict(x_test)

In [30]:
accuracy_lgbm = mean_squared_error(ypred2,y_test)
accuracy_lgbm

2.451352924200758

In [31]:
r2_score(ypred2,y_test)

0.9592684612399517